# Epidemic visualization demos

Epivislab can ingest simulation outputs from Episimlab. The outputs must be saved in a format that can be loaded as an `xarray`.

Epivislab provides a flexible interface for summarizing and aggregating simulations along various axes. Epivislab recognizes four main categories of dimensions defined in simulation xarrays:

- The **State** dimension enumerates the disease compartment states in the model (e.g. "susceptible", "infectious") and should be found in each simulation.
- The **Time** dimension defines the ordering of steps in each simulation.
- The remaining **Within-simulation** dimensions include any axes that are present in the contact matrix (e.g. "age", "risk"). These dimensions should be found in each individual simulation.
- The **Between-simulation** dimensions are those that distinguish different replicates in the same set of simulations. The simplest implementation is an index that indicates which replicate a particular data point belongs to.
- Finally, the **Measured values** are the numeric outputs from the simuluation -- the population size in each compartment.

Epivislab integrates with `dask` to provide parallel computation of summary statistics for large simulations, and uses `plotly` to render interactive visuals based on those aggregations.

In [1]:
import xarray as xr
import numpy as np
import pandas as pd
import dask.dataframe as dd
from epivislab.stats import Quantile
from epivislab.simhandler import EpiSummary

Simulations from episimlab produce xarrays, and those data objects should be saved in a file format that supports this data structure. For this example, we'll use a small simulation saved in the `zarr` format.

In [2]:
sims = xr.open_zarr('/Users/kpierce/epimodels/epimodels/outputs/test_sim_2.zarr/')

The xarray data structure contains helpful metadata that we can use to understand the parameters of the simulation. The coordinates and dimensions of the xarray tell us about the structure of the model and the contact matrix.

In [3]:
sims.coords

Coordinates:
  * age      (age) object '0-4' '5-17' '18-49' '50-64' '65+'
  * compt    (compt) object 'S' 'E' 'Pa' 'Py' 'Ia' 'Iy' 'Ih' 'R' 'D'
  * index    (index) int64 0 1 2 3 4 5 6 7 8 9
  * risk     (risk) object 'low' 'high'
  * step     (step) datetime64[ns] 2020-03-11 2020-03-12 ... 2020-04-01
  * vertex   (vertex) object 'Austin'

The keys of the xarray tell us about the values in the xarray:

In [4]:
list(sims.keys())

['compt_model__state',
 'rate_E2Pa__tau',
 'rate_E2Py__tau',
 'rate_Iy2Ih__eta',
 'rate_S2E__beta',
 'setup_gamma_Ia__tri_Iy2R_para',
 'setup_gamma_Ih__tri_Ih2R',
 'setup_mu__tri_Ih2D',
 'setup_rho_Ia__tri_Pa2Ia',
 'setup_rho_Iy__tri_Py2Iy',
 'setup_seed__seed_entropy',
 'setup_sigma__tri_exposed_para',
 'setup_sto__sto_toggle']

## High-level visualization API

### `EpiSummary` objects

To process a simulation output for visualization, we first need to create an `EpiSummary` object and identify the state, time, within-simulation, between-simulation, and measured coordinates in our xarray. These attributes will allow `EpiSummary` methods to parse the xarray and produce aggregate statistics.

In [5]:
test = EpiSummary(
    simulation=sims,
    state_coord=['compt'],
    within_sim_coord=['age', 'risk', 'vertex'],
    time_coord=['step'],
    between_sim_coord=['index'],
    measured_coord=['compt_model__state']
)

### Prediction interval plots

The `EpiSummary.interval_plot()` method calculates the median and upper and lower quantiles, and generates time series graphs that display simulation median and spread between upper and lower quantiles.

The grouping variables -- state, time, and within-simulation coordinates -- indicate within which groups to calculate median and quantiles. The measurement variable must also be specified, as well as the values of the quantiles to calculate.

The resulting plot uses plotly widgets to allow selection of specific groups for display. Dropdown menus are populated with the values of each grouping coordinate.

In [6]:
fig = test.interval_plot(
    groupers=['age', 'risk', 'step', 'vertex', 'compt'], 
    aggcol='compt_model__state',
    upper=0.9, 
    lower=0.05)

Calculating quantile 0.5 for ['compt_model__state'] after summation over variables [None].
Dropping columns Index(['index'], dtype='object') and aggregating by ['age', 'risk', 'step', 'vertex', 'compt'].


/Users/kpierce/epivislab/evl-venv/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:1231: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  v = np.array(v, copy=False)
/Users/kpierce/epivislab/evl-venv/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:1625: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  subarr = np.array(values, dtype=dtype, copy=copy)
/Users/kpierce/epivislab/evl-venv/lib/python3.9/site-packages/pandas/core/common.py:228: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of li

Calculating quantile 0.9 for ['compt_model__state'] after summation over variables [None].
Dropping columns Index(['index'], dtype='object') and aggregating by ['age', 'risk', 'step', 'vertex', 'compt'].
Calculating quantile 0.05 for ['compt_model__state'] after summation over variables [None].
Dropping columns Index(['index'], dtype='object') and aggregating by ['age', 'risk', 'step', 'vertex', 'compt'].


In [7]:
fig

If not all within-simulation coordinates are passed as grouping variables, the results will be summed over that coordinate prior to median and quantile calculation. In the example below, `age` is omitted from the `groupers` argument, so populations within each vertex, risk group, and vertex are summed (the `prediction_interval` understand that `step` and `compt` are special coordinates and knows not to sum over them). The dropdown menus reflect only the grouping coordinates retained after the summation.

In [8]:
fig2 = test.interval_plot(
    groupers=['risk', 'step', 'vertex', 'compt'], 
    aggcol='compt_model__state',
    upper=0.9, 
    lower=0.05)

Summing ['compt_model__state'] over variables {'age'}; retaining groups ['risk', 'step', 'vertex', 'compt', 'index'].
Calculating quantile 0.5 for ['compt_model__state'] after summation over variables [{'age'}].
Dropping columns Index([], dtype='object') and aggregating by ['risk', 'step', 'vertex', 'compt'].
Summing ['compt_model__state'] over variables {'age'}; retaining groups ['risk', 'step', 'vertex', 'compt', 'index'].
Calculating quantile 0.9 for ['compt_model__state'] after summation over variables [{'age'}].

/Users/kpierce/epivislab/evl-venv/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:1231: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.

/Users/kpierce/epivislab/evl-venv/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:1625: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.

/Users/kpierce/epivislab/evl-venv/lib/python3.9/site-packages/pandas/core/common.py:228: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If


Dropping columns Index([], dtype='object') and aggregating by ['risk', 'step', 'vertex', 'compt'].
Summing ['compt_model__state'] over variables {'age'}; retaining groups ['risk', 'step', 'vertex', 'compt', 'index'].
Calculating quantile 0.05 for ['compt_model__state'] after summation over variables [{'age'}].
Dropping columns Index([], dtype='object') and aggregating by ['risk', 'step', 'vertex', 'compt'].


In [9]:
fig2

### Spaghetti plots

The `EpiSummary.spaghetti_plot()` generates a time series with individual simulation results represented as different lines.

By default, no aggregations or calculations are performed on simulation data to generate this plot. Simply calling the `EpiSummary.spaghetti_plot()` method will generate a plot with one line per simulation.

In [10]:
fig3 = test.spaghetti_plot()

In [11]:
fig3

Optionally, you may specify a set of grouping variables. As with the `EpiSummary.prediction_interval()` method, simulation results will be summed over coordinates not listed as grouping variables and the dropdown menus are updated accordingly. An `aggcol` must be specified in `groupers` are listed.

In [12]:
fig4 = test.spaghetti_plot(groupers=['age', 'compt', 'vertex', 'index', 'step'], aggcol='compt_model__state')

Not all compt_model__state measures are listed as simulation measurements (['compt_model__state']).
Summing compt_model__state over variables {'risk'}; retaining groups ['age', 'compt', 'vertex', 'index', 'step'].


In [13]:
fig4

## Other `EpiSummary` methods

`EpiSummary.interval_plot()` and `EpiSummary.spaghetti_plot()` are high-level workflow methods that manipulate data and generate plots without returning data.

Other methods in the `EpiSummary` class manipulate data and return data objects.
- `EpiSummary.sum_over_groups()` sums xarray data within individual simulations, maintaining a specified set of data groupings, and returns a dask DataFrame.
- `EpiSummary.quantile_between_sims()` calculates quantiles for variables (grouped or ungrouped) across multiple simulations in an xarray, and returns a dask DataFrame. If any variables are missing from the list of grouping variables, this method will call `EpiSummary.sum_over_groups()` to sum them before calculating quantiles. This ensures a valid quantile calculation.
- `EpiSummary.prediction_interval()` calculates the median, upper and lower quantiles of xarray data and returns n xarray.

### Prediction interval workflow

Submodule `epivislab.timeseries` contains methods for generating prediction intervals and spaghetti plots from xarray data.

In [14]:
from epivislab.timeseries import interval_timeseries, spaghetti_timeseries

The method `Episimlab.prediction_interval()` calculates the median, upper, and lower quantiles and returns a new `xarray` object.

In [15]:
summary_all_ages = test.prediction_interval(
    groupers=['risk', 'step', 'vertex', 'compt'], 
    aggcol='compt_model__state',
    upper=0.9, 
    lower=0.05)

Summing ['compt_model__state'] over variables {'age'}; retaining groups ['risk', 'step', 'vertex', 'compt', 'index'].
Calculating quantile 0.5 for ['compt_model__state'] after summation over variables [{'age'}].
Dropping columns Index([], dtype='object') and aggregating by ['risk', 'step', 'vertex', 'compt'].


/Users/kpierce/epivislab/evl-venv/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:1231: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.

/Users/kpierce/epivislab/evl-venv/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:1625: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.

/Users/kpierce/epivislab/evl-venv/lib/python3.9/site-packages/pandas/core/common.py:228: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If

Summing ['compt_model__state'] over variables {'age'}; retaining groups ['risk', 'step', 'vertex', 'compt', 'index'].
Calculating quantile 0.9 for ['compt_model__state'] after summation over variables [{'age'}].
Dropping columns Index([], dtype='object') and aggregating by ['risk', 'step', 'vertex', 'compt'].
Summing ['compt_model__state'] over variables {'age'}; retaining groups ['risk', 'step', 'vertex', 'compt', 'index'].
Calculating quantile 0.05 for ['compt_model__state'] after summation over variables [{'age'}].
Dropping columns Index([], dtype='object') and aggregating by ['risk', 'step', 'vertex', 'compt'].


In [16]:
summary_all_ages

<xarray.Dataset>
Dimensions:  (risk: 2, step: 22, vertex: 1, compt: 9)
Coordinates:
  * risk     (risk) object 'high' 'low'
  * step     (step) datetime64[ns] 2020-03-11 2020-03-12 ... 2020-04-01
  * vertex   (vertex) object 'Austin'
  * compt    (compt) object 'D' 'E' 'Ia' 'Ih' 'Iy' 'Pa' 'Py' 'R' 'S'
Data variables:
    index_x  (risk, step, vertex, compt) int64 0 1 2 3 4 ... 391 392 393 394 395
    upper    (risk, step, vertex, compt) float64 0.0 0.0 ... 979.0 1.421e-15
    index_y  (risk, step, vertex, compt) int64 0 1 2 3 4 ... 391 392 393 394 395
    lower    (risk, step, vertex, compt) float64 0.0 0.0 25.0 ... 0.45 943.5 0.0
    index    (risk, step, vertex, compt) int64 0 1 2 3 4 ... 391 392 393 394 395
    median   (risk, step, vertex, compt) float64 0.0 0.0 25.0 ... 2.0 963.0 0.0

The `summary_all_ages` object can be passed to the `interval_timeseries()` plot to generate the plotly image.

In [17]:
interval_timeseries(summary_all_ages)

Spaghetti plots do not require additonal calculations, so they can also be generated directly from the simulation `xarray` without building an `EpiSummary` object first:

In [18]:
spaghetti_timeseries(
    simulation_xr=sims,
    x_val='step',
    y_val='compt_model__state',
    index_coord='index'
)

## Low level API

At the lowest level, `epivislab.stats` implements aggregation methods for dask DataFrames. These methods are wrappers on the `dask.DataFrame.groupby()` and `dask.DataFrame.Aggregate()` methods.

Class `Quantile` calculates a user-specified quantile:

In [19]:
from epivislab.stats import Sum, Quantile

In [20]:
sims_dd = sims.to_dask_dataframe()

In [21]:
sims_quantile = Quantile(quantile=0.8)
sims_q80 = sims_quantile.dd_quantile(
    ddf=sims_dd, 
    groupers=['age', 'risk', 'step', 'compt', 'vertex'], 
    aggcol='compt_model__state'
)

Dropping columns Index(['index', 'rate_E2Pa__tau', 'rate_E2Py__tau', 'rate_Iy2Ih__eta',
       'rate_S2E__beta', 'setup_gamma_Ia__tri_Iy2R_para',
       'setup_gamma_Ih__tri_Ih2R', 'setup_mu__tri_Ih2D',
       'setup_rho_Ia__tri_Pa2Ia', 'setup_rho_Iy__tri_Py2Iy',
       'setup_seed__seed_entropy', 'setup_sigma__tri_exposed_para',
       'setup_sto__sto_toggle', 'value'],
      dtype='object') and aggregating by ['age', 'risk', 'step', 'compt', 'vertex'].


/Users/kpierce/epivislab/evl-venv/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:1231: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.

/Users/kpierce/epivislab/evl-venv/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:1625: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.

/Users/kpierce/epivislab/evl-venv/lib/python3.9/site-packages/pandas/core/common.py:228: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If

In [22]:
sims_q80.compute()

,value,age,risk,step,compt,vertex
0,0.000000,0-4,high,2020-03-11,D,Austin
1,0.000000,0-4,high,2020-03-11,E,Austin
2,5.000000,0-4,high,2020-03-11,Ia,Austin
3,0.000000,0-4,high,2020-03-11,Ih,Austin
4,0.000000,0-4,high,2020-03-11,Iy,Austin
...,...,...,...,...,...,...
1975,14.400000,65+,low,2020-04-01,Iy,Austin
1976,1.066667,65+,low,2020-04-01,Pa,Austin
1977,1.000000,65+,low,2020-04-01,Py,Austin
1978,187.600000,65+,low,2020-04-01,R,Austin


Class `Sum` calculates summations:

In [23]:
sims_sum = Sum()
sims_sum_over_risk = sims_quantile.dd_quantile(
    ddf=sims_dd, 
    groupers=['age', 'step', 'compt', 'vertex'], 
    aggcol='compt_model__state'
)

Dropping columns Index(['index', 'rate_E2Pa__tau', 'rate_E2Py__tau', 'rate_Iy2Ih__eta',
       'rate_S2E__beta', 'risk', 'setup_gamma_Ia__tri_Iy2R_para',
       'setup_gamma_Ih__tri_Ih2R', 'setup_mu__tri_Ih2D',
       'setup_rho_Ia__tri_Pa2Ia', 'setup_rho_Iy__tri_Py2Iy',
       'setup_seed__seed_entropy', 'setup_sigma__tri_exposed_para',
       'setup_sto__sto_toggle', 'value'],
      dtype='object') and aggregating by ['age', 'step', 'compt', 'vertex'].


/Users/kpierce/epivislab/evl-venv/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:1231: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.

/Users/kpierce/epivislab/evl-venv/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:1625: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.

/Users/kpierce/epivislab/evl-venv/lib/python3.9/site-packages/pandas/core/common.py:228: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If

In [24]:
sims_sum_over_risk.compute()

,value,age,step,compt,vertex
0,0.000000,0-4,2020-03-11,D,Austin
1,0.000000,0-4,2020-03-11,E,Austin
2,5.000000,0-4,2020-03-11,Ia,Austin
3,0.000000,0-4,2020-03-11,Ih,Austin
4,0.000000,0-4,2020-03-11,Iy,Austin
...,...,...,...,...,...
985,16.000000,65+,2020-04-01,Iy,Austin
986,1.066667,65+,2020-04-01,Pa,Austin
987,1.000000,65+,2020-04-01,Py,Austin
988,186.200000,65+,2020-04-01,R,Austin
